# Qwen2.5-7B模型微调流程
本notebook参考Qwen2_5_7B_Alpaca_fintune.ipynb，使用已处理好的QA数据进行微调。

In [1]:
# 1. 安装和导入依赖库
# 如果未安装可取消注释运行
# !pip install unsloth bitsandbytes accelerate xformers==0.0.29.post3 peft trl sentencepiece protobuf datasets huggingface_hub hf_transfer
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/xiaoke/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


## 2. 加载和配置Qwen2.5-7B模型
设置最大序列长度、数据类型和量化方式，加载预训练模型和分词器。

In [2]:
# 2. 加载和配置Qwen2.5-7B模型
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # 如需访问受限模型请填写token
)

==((====))==  Unsloth 2025.7.11: Fast Qwen2 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 44.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.41s/it]



## 3. 添加LoRA适配器
为模型添加LoRA适配器，只微调部分参数以节省显存和加速训练。

In [3]:
# 3. 添加LoRA适配器
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.7.11 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## 4. 加载QA数据集
加载已处理好的QA数据集（qa_train.json），用于微调。

In [4]:
# 4. 加载QA数据集
qa_data_path = "./datas/qa_train.json"
import json
from datasets import Dataset
with open(qa_data_path, "r", encoding="utf-8") as f:
    qa_data = json.load(f)
dataset = Dataset.from_list(qa_data)
print(f"QA样本数: {len(dataset)}")
print("示例样本:", dataset[0])

QA样本数: 118
示例样本: {'instruction': '你是一名专业的财报数据问答助手', 'input': '根据联邦制药（03933.HK）的公司发展历程，请简述其在2023年的重大产品临床进展。', 'output': '根据联邦制药（03933.HK）的公司发展历程，2023年的重大产品临床进展如下：\n\n- **重磅产品临床**：\n  - UBT251多个适应症进入临床阶段；\n  - 司美格鲁肽注射液体重管理适应症获临床受理；\n  - 利拉鲁肽注射液BLA；德谷胰岛素利拉鲁肽注射液获批临床。'}


## 5. 格式化数据并添加EOS标记
定义格式化函数，将指令、输入和输出拼接为训练文本，并在末尾添加EOS标记。

In [5]:
# 5. 格式化数据并添加EOS标记
qa_prompt = """以下是一条描述任务的指令，配有进一步的输入信息。请根据要求完成回复。\n\n### 指令:\n{}\n\n### 输入:\n{}\n\n### 回复:\n{}"""
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = qa_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}
dataset = dataset.map(formatting_prompts_func, batched=True)

Map: 100%|██████████| 118/118 [00:00<00:00, 19510.70 examples/s]



## 6. 训练模型
使用TRL的SFTTrainer进行微调，设置训练参数如批次大小、学习率、训练步数等。

In [ ]:
# 6. 训练模型
from trl import SFTConfig, SFTTrainer

# SFTTrainer用于有监督微调（Supervised Fine-Tuning），可高效训练大模型
trainer = SFTTrainer(
    model = model,                      # 传入待微调的模型
    tokenizer = tokenizer,              # 分词器，用于文本编码
    train_dataset = dataset,            # 训练数据集，已格式化为"text"字段
    dataset_text_field = "text",        # 指定训练文本字段名
    max_seq_length = max_seq_length,    # 最大序列长度，影响显存和训练速度
    packing = False,                    # 是否启用packing（将多个短文本拼接为一个长序列以加速训练），此处关闭
    args = SFTConfig(                   # 训练参数配置
        per_device_train_batch_size = 2,        # 每张GPU上的训练批次大小
        gradient_accumulation_steps = 4,        # 梯度累积步数，等效于总batch_size=2*4=8
        warmup_steps = 5,                       # 预热步数，前5步线性增加学习率
        max_steps = 60,                         # 最大训练步数（可根据数据量和显存调整）
        learning_rate = 2e-4,                   # 学习率
        logging_steps = 1,                      # 日志打印频率，每步打印一次
        optim = "adamw_8bit",                   # 优化器类型，采用8bit AdamW以节省显存
        weight_decay = 0.01,                    # 权重衰减，防止过拟合
        lr_scheduler_type = "linear",           # 学习率调度策略，线性下降
        seed = 3407,                            # 随机种子，保证实验可复现
        output_dir = "spark_model_train_outputs",# 输出目录，保存训练结果和模型
        report_to = "none",                     # 日志上报平台（如wandb），此处关闭
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 118/118 [00:01<00:00, 107.92 examples/s]



## 7. 显存信息监控
通过torch.cuda获取和打印显卡显存使用情况，便于监控资源消耗。

In [7]:
# 7. 显存信息监控
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU型号: {gpu_stats.name}，最大显存: {max_memory} GB")
print(f"已预留显存: {start_gpu_memory} GB")

GPU型号: NVIDIA RTX A6000，最大显存: 44.988 GB
已预留显存: 7.246 GB


## 8. 启动训练并监控显存变化
开始训练模型，并显示训练结果和显存变化。

In [8]:
# 8. 启动训练并监控显存变化
trainer_stats = trainer.train()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"训练耗时: {trainer_stats.metrics['train_runtime']} 秒")
print(f"训练耗时: {round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟")
print(f"训练期间峰值显存: {used_memory} GB，占最大显存 {used_percentage}%")
print(f"LoRA训练显存: {used_memory_for_lora} GB，占最大显存 {lora_percentage}%")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 118 | Num Epochs = 4 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.153800
2,2.293200
3,2.277500
4,2.183100
5,2.087200
6,2.189800
7,1.788600
8,1.591500
9,1.673200
10,1.491400


训练耗时: 164.8592 秒
训练耗时: 2.75 分钟
训练期间峰值显存: 8.092 GB，占最大显存 17.987%
LoRA训练显存: 0.846 GB，占最大显存 1.881%


## 9. 推理与保存微调后的模型
训练完成后，进行简单推理验证，并保存LoRA适配器和分词器。

In [9]:
# 9. 推理与保存微调后的模型
FastLanguageModel.for_inference(model)  # 开启高效推理模式
test_instruction = "你是一名专业的财报数据问答助手"
test_input = "联邦制药2024年营业收入和净利润分别是多少？"
test_prompt = qa_prompt.format(test_instruction, test_input, "")
inputs = tokenizer([test_prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
print("推理结果:", tokenizer.batch_decode(outputs))


推理结果: ['以下是一条描述任务的指令，配有进一步的输入信息。请根据要求完成回复。\n\n### 指令:\n你是一名专业的财报数据问答助手\n\n### 输入:\n联邦制药2024年营业收入和净利润分别是多少？\n\n### 回复:\n根据图片中的图表和文字内容，联邦制药2024年营业收入为100.1亿元，净利润为20.1亿元。<|endoftext|>']


In [ ]:

# 保存LoRA适配器和分词器到本地
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")